### Generate dataset to be passed to disnet

First import the classifier to classify traffic sign shape

In [1]:
classifier = 'shape-classifier-aug.keras'

from tensorflow import keras
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)
        
        
model = keras.models.load_model(classifier)
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
f

### Create helper functions

define paths for labels and images for train, val and test sets \
Functions to create masks and vectors from label

In [2]:
#imports
import math
import glob
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2

#construct val, train or test data
def select_dataset(data):
    
    yolo_path = r'C:\Users\portierl4527\ARCADIS\Afstuderen Levi - Depth estimation - General\yolov5\yolov5-master\runs\detect'
    img_path= r'C:\Users\portierl4527\ARCADIS\Afstuderen Levi - Depth estimation - General\Cityscapes-Sequence\leftImg8bit_sequence_trainvaltest'
    
    lab_path = os.path.join(yolo_path, data+'\labels')
    img_path = os.path.join(img_path, data)
   
    labels = glob.glob(lab_path+'\*.txt')
#     images = glob.glob(img_path+'\*.png')
    
    return labels, img_path


#ground truth file
GT_PATH = r'C:\Users\portierl4527\ARCADIS\Afstuderen Levi - Depth estimation - General\Cityscapes-Sequence\disparity_sequence_trainvaltest\gt_depth'
gt_labels = glob.glob(GT_PATH+'\*.npy')


def transform2vector(class_id, x, y, w, h):
    'takes yolo label as input and returns vector for disnet'
    
    dbbox = math.sqrt(w*w + h*h)
    d= 1/(dbbox) #diagonal of bounding box /2289.7
    w = 1/(w) #widht is 1/ (width/img size)
    h = 1/(h) #height is 1/ (height/img size)
     
    class_id = int(class_id)
    if class_id == 0:
        cw = 60 #traffic signs width
        ch = 60
        cd = 5
    else:
        cw = 30
        ch = 70 #traffic light height
        cd = 25
    
    return class_id, w, h, d, cw, ch, cd


def mask(x, y, w, h):
    "takes yolo label and creates mask of object"
    
    width = 2048 #image size
    height = 1024
    
    xmax = int((x*width) + (w * width)/2.0)
    xmin = int((x*width) - (w * width)/2.0)
    ymax = int((y*height) + (h * height)/2.0)
    ymin = int((y*height) - (h * height)/2.0)
    
    return xmin, xmax, ymin, ymax


In [13]:
#set labels to train
labels, IMAGE_PATH = select_dataset('low_quality')

filename = []
y_true = []
object_id = []
vector = []    
bbox = []
shape = []
o_width = []
o_height = []
 
#for txt file in yolo labels    
for file in tqdm(labels):
    label_norm = np.loadtxt(file).reshape(-1, 5)
    
    basename = os.path.basename(file) #filename
    basename_no_ext = os.path.splitext(basename)[0]
    #[:-12]
   
    #for line in txt file
    for i in range(len(label_norm)):
        labels_conv = label_norm[i]
        class_id, w, h, d, cw, ch, cd = transform2vector(labels_conv[0], labels_conv[1], labels_conv[2], labels_conv[3], labels_conv[4])
        xmin, xmax, ymin, ymax =  mask(labels_conv[1], labels_conv[2], labels_conv[3], labels_conv[4])
        
        object_id.append(class_id) #append classID
        #vector.append(np.array([w,h,d,cw,ch,cd])) #create the vector
        filename.append(basename_no_ext) #add_filename
        bbox.append([xmin, xmax, ymin, ymax])
        o_width.append(xmax-xmin)
        o_height.append(ymax-ymin)
        
        #if object is a traffic sign
        if class_id == 0:
            
            #find corresponding image
            img = os.path.join(IMAGE_PATH,'{}.png'.format(basename_no_ext))
            img_array = cv2.imread(img, cv2.IMREAD_GRAYSCALE)#read image in grayscale
            img_array = img_array[ymin:ymax,xmin:xmax] #mask the object

            new_array = cv2.resize(img_array, (28,28)) #resize the object
            X = np.array(new_array).reshape(-1, 28, 28, 1) #reshape the object

            pred_shape = model.predict_classes(X) #predict the class
            shape.append(pred_shape) #add to list

            if pred_shape == 4: 
                vector.append(np.array([w,h,d,76,76,5])) #triangular signs
            elif pred_shape == 3:
                vector.append(np.array([w,h,d,51,51,5])) #square signs
            else:
                vector.append(np.array([w,h,d,52,52,5])) #other signs


        #if object is a traffic light        
        else:
            vector.append(np.array([w,h,d,cw,ch,cd]))
            shape.append(99)
                
        #search for corresponding ground truth file
        gt = os.path.join(GT_PATH,'{}_disparity.npy'.format(basename_no_ext[:-4]))      
        object_gt = np.load(gt)
            
        y_true.append(np.median(object_gt[ymin:ymax,xmin:xmax])) #add the true distance

100%|██████████████████████████████████████████████████████████████████████████████| 1342/1342 [02:51<00:00,  7.84it/s]


In [14]:
#create dataframe
data = {'filename': filename,
       'object_id': object_id,
       'y_true': y_true,
       'vectors': vector,
       'o_width': o_width,
       'o_height': o_height,
       'shape': shape,
       'bbox': bbox}

df = pd.DataFrame(data=data)

#only keep smaller than 60 meters
df = df[df.y_true < 60.0]

#drop wrong traffic signs
wrong_size = df[ (df.object_id == 0) & (df.o_width > 2*df.o_height) ].index
also_wrong_size = df[ (df.object_id == 0) & (df.o_height > 2*df.o_width) ].index

df.drop(wrong_size, inplace=True)
df.drop(also_wrong_size, inplace=True)


#df.to_pickle('train_data4disnet.pkl')


#df.to_pickle('test_data4disnet.pkl')


df.to_pickle('low_data4disnet.pkl')
    
df.head()

,filename,object_id,y_true,vectors,o_width,o_height,shape,bbox
0,bonn_000000_000001_low,0,36.790001,"[73.14272339616294, 42.666666666666664, 36.854...",28,24,[0],"[1141, 1169, 350, 374]"
1,bonn_000000_000002_low,0,35.660000,"[75.85181589247246, 39.38465416335179, 34.9537...",27,26,[0],"[1145, 1172, 346, 372]"
2,bonn_000000_000008_low,0,30.040001,"[68.26684142978073, 34.133304206247075, 30.529...",30,31,[2],"[1190, 1220, 288, 319]"
3,bonn_000000_000009_low,0,29.610001,"[66.0645979638891, 32.0, 28.7994091416293, 52....",31,32,[0],"[1196, 1227, 284, 316]"
4,bonn_000000_000012_low,0,26.370001,"[58.51443551124063, 30.117669735657213, 26.778...",35,34,[2],"[1227, 1262, 267, 301]"


In [24]:
#create plots
import matplotlib.pyplot as plt

#df containing signs and lights
sign_data = df[df.object_id == 0]
light_data  = df[df.object_id == 1] 

#index of rows
signs  = df.index[df['object_id'] == 0].to_list()
lights = df.index[df['object_id'] == 1].to_list()

#all widths, heights and diagonals
width = [vector[i][0] for i in range(len(vector))]
height = [vector[i][1] for i in range(len(vector))]
diagonal = [vector[i][2] for i in range(len(vector))]


plt.scatter(x=df.o_height, y=df.absolute_error)
#plt.scatter(x=df.y_true, y=(diagonal))
#plt.scatter(x=light_data.y_true, y=(light_w), label = 'lights')
plt.title('Distance vs 1/width')
plt.show()


AttributeError: 'DataFrame' object has no attribute 'absolute_error'